<a href="https://colab.research.google.com/github/deninakovska/Dizajn_i_arhitektura_na_softver/blob/main/Domasna1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 7.5 MB/s eta 0:00:00


In [ ]:
from bs4 import BeautifulSoup
import requests
import re

url = 'https://www.mse.mk/mk/stats/symbolhistory/REPL'
names = []
filtered_names = []

def publisher_codes(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    options = soup.find_all('option')

    for option in options:
        name = option.text.strip()

        if name and len(name) > 1 and re.match(r'^[^\d]*$', name):
            names.append(name)

    for name in names:
        filtered = {
            "Publisher code": name
        }
        filtered_names.append(filtered)

    return filtered_names

filtered_names = publisher_codes(url)



In [ ]:
import pandas as pd
df = pd.DataFrame(filtered_names)
df

,Publisher code
0,ADIN
1,ALK
2,ALKB
3,AMEH
4,APTK
...,...
159,ZIMS
160,ZKAR
161,ZPKO
162,ZPOG


In [ ]:
df.to_csv('data.csv', index=False)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

def check_for_existing_data(code):
  data = pd.read_csv(f'{code}.csv')
  if not data['Датум'].empty:
    data['Датум'] = pd.to_datetime(data['Датум'], format='%d.%m.%Y')
    return data['Датум'].max()

  return None

def add_data(code, new_data):
    filename = f"{code}.csv"
    new_df = pd.DataFrame(new_data)
    new_df.to_csv(filename, mode='a', header=not pd.io.common.file_exists(filename), index=False)

In [ ]:
def scroll():
          last_row_count = 0
          same_row_count_counter = 0
          max_same_count_threshold = 3

          while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(3)

            rows = driver.find_elements(By.XPATH, "//tbody/tr")
            new_row_count = len(rows)

            if new_row_count == last_row_count:
                same_row_count_counter += 1
            else:
                same_row_count_counter = 0

            last_row_count = new_row_count

            if same_row_count_counter >= max_same_count_threshold:
                break

          return rows


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from datetime import date
from selenium.webdriver.common.action_chains import ActionChains

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)


columns = ['Датум', 'Цена на последна трансакција', 'Макс.', 'Мин.', 'Просечна цена', '%пром.', 'Количина',
           'Промет во БЕСТ во денари', 'Вкупен промет во денари']


def filter2(kodovi):
    for i in range(len(filtered_names)):
        current = kodovi[i]['Publisher code']
        new_url = f"https://www.mse.mk/mk/stats/symbolhistory/{current}"
        driver.get(new_url)


        new_file = pd.DataFrame(columns=columns)
        filename = new_file.to_csv(f'{current}.csv', index=False)

        if not check_for_existing_data(current):
            for j in range(10, 0, -1):
                today = date.today()
                ten_years_ago = today.replace(year=today.year - j)
                ten_years_ago_str = ten_years_ago.strftime("%d.%m.%Y")
                until = today.replace(year=today.year - (j - 1))
                until_str = until.strftime("%d.%m.%Y")
                for attempt in range(3):
                    try:
                        from_date = WebDriverWait(driver, 5).until(
                            EC.element_to_be_clickable((By.ID, 'FromDate'))
                        )
                        driver.execute_script("arguments[0].scrollIntoView();", from_date)
                        from_date.clear()
                        from_date.send_keys(ten_years_ago_str + Keys.RETURN)

                        until_date = WebDriverWait(driver, 5).until(
                            EC.element_to_be_clickable((By.ID, 'ToDate'))
                        )
                        driver.execute_script("arguments[0].scrollIntoView();", until_date)
                        until_date.clear()
                        until_date.send_keys(until_str + Keys.RETURN)

                        submit_button = WebDriverWait(driver, 5).until(
                            EC.element_to_be_clickable((By.XPATH, '//input[@type="submit" and @value="Прикажи"]'))
                        )
                        driver.execute_script("arguments[0].click();", submit_button)



                        WebDriverWait(driver, 5).until(
                            EC.presence_of_all_elements_located((By.XPATH, "//tbody/tr"))
                        )

                        all_rows = scroll()

                        data_to_add = []

                        for row in all_rows:
                            cells = row.find_elements(By.TAG_NAME, 'td')
                            if len(cells) >= 9:
                                datum = cells[0].text.strip()
                                maks = cells[1].text.strip()
                                minn = cells[2].text.strip()
                                prosecna_cena = cells[3].text.strip()
                                posledna_transakcija = cells[4].text.strip()
                                prom = cells[5].text.strip()
                                kolicina = cells[6].text.strip()
                                promet_denari = cells[7].text.strip()
                                vkupen_promet_denari = cells[8].text.strip()

                                if maks and minn:
                                    data_to_add.append({
                                        'Датум': datum,
                                        'Цена на последна трансакција': posledna_transakcija,
                                        'Макс.': maks,
                                        'Мин.': minn,
                                        'Просечна цена': prosecna_cena,
                                        '%пром.': prom,
                                        'Количина': kolicina,
                                        'Промет во БЕСТ во денари': promet_denari,
                                        'Вкупен промет во денари': vkupen_promet_denari
                                    })

                        print(data_to_add)
                        add_data(current, data_to_add)

                        break

                    except Exception as e:

                            continue


                driver.refresh()
                time.sleep(2)

filter2(publisher_codes(url))

datum za period 10.11.2014 - 10.11.2015, code:ADIN
[{'Датум': '10.11.2015', 'Цена на последна трансакција': '290,00', 'Макс.': '290,00', 'Мин.': '290,00', 'Просечна цена': '290,00', '%пром.': '0,00', 'Количина': '200', 'Промет во БЕСТ во денари': '58.000', 'Вкупен промет во денари': ''}, {'Датум': '05.11.2015', 'Цена на последна трансакција': '290,00', 'Макс.': '290,00', 'Мин.': '290,00', 'Просечна цена': '290,00', '%пром.': '7,41', 'Количина': '100', 'Промет во БЕСТ во денари': '29.000', 'Вкупен промет во денари': ''}, {'Датум': '04.11.2015', 'Цена на последна трансакција': '270,00', 'Макс.': '270,00', 'Мин.': '270,00', 'Просечна цена': '270,00', '%пром.': '-9,70', 'Количина': '1.167', 'Промет во БЕСТ во денари': '315.090', 'Вкупен промет во денари': ''}, {'Датум': '30.10.2015', 'Цена на последна трансакција': '299,00', 'Макс.': '299,00', 'Мин.': '299,00', 'Просечна цена': '299,00', '%пром.': '0,00', 'Количина': '50', 'Промет во БЕСТ во денари': '14.950', 'Вкупен промет во денари': ''

KeyboardInterrupt: 